In [10]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import os
import skimage.io
import glob
from tqdm import tqdm
from skimage.io import imread, imshow
from skimage.transform import resize
from sklearn.utils import shuffle
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import InputLayer, Conv2D, BatchNormalization, MaxPool2D, Dropout, Flatten, Dense, GlobalAveragePooling2D
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [11]:
train_dataset_0_all = glob.glob('./C-NMC_Leukemia/training_data/fold_0/all/*.bmp')
train_dataset_0_hem = glob.glob('./C-NMC_Leukemia/training_data/fold_0/hem/*.bmp')
train_dataset_1_all = glob.glob('./C-NMC_Leukemia/training_data/fold_1/all/*.bmp')
train_dataset_1_hem = glob.glob('./C-NMC_Leukemia/training_data/fold_1/hem/*.bmp')
train_dataset_2_all = glob.glob('./C-NMC_Leukemia/training_data/fold_2/all/*.bmp')
train_dataset_2_hem = glob.glob('./C-NMC_Leukemia/training_data/fold_2/hem/*.bmp')

valid_data = pd.read_csv('./C-NMC_Leukemia/validation_data/C-NMC_test_prelim_phase_data_labels.csv')


In [12]:
A = []
H = []

A.extend(train_dataset_0_all)
A.extend(train_dataset_1_all)
A.extend(train_dataset_2_all)

H.extend(train_dataset_0_hem)
H.extend(train_dataset_1_hem)
H.extend(train_dataset_2_hem)

A = np.array(A)
H = np.array(H)

In [13]:
Image = []
Label = []


In [14]:
for i in tqdm(range(0, len(A))):
    img = imread(A[i])
    img = resize(img, (128, 128))
    Image.append(img)
    Label.append(1)
for i in tqdm(range(0, len(H))):
    img = imread(H[i])
    img = resize(img, (128, 128))
    Image.append(img)
    Label.append(0)

100%|██████████| 3389/3389 [01:12<00:00, 46.76it/s]


In [15]:
Image = np.array(Image)
Label = np.array(Label)

In [16]:
del A
del H


In [17]:
X = Image
y = Label

In [18]:
del Image
del Label

In [19]:
X_val = []

for image_name in valid_data.new_names:
    img = imread('./C-NMC_Leukemia/validation_data/C-NMC_test_prelim_phase_data/' + image_name)
    img = resize(img, (128, 128))
    X_val.append(img)

In [20]:
X_val = np.array(X_val)
y_val = valid_data.labels.values

In [21]:
train_datagen = ImageDataGenerator(horizontal_flip=True, vertical_flip=True, zoom_range=0.2)
train_datagen.fit(X)

In [22]:
base_model = tf.keras.applications.MobileNetV2(weights='imagenet', include_top=False, input_shape=(128, 128, 3))

for layer in base_model.layers:
    layer.trainable = False

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(1, activation='sigmoid')(x)

9406464/9406464 [==============================] - 2s 0us/step


In [23]:
model = Model(inputs=base_model.input, outputs=predictions)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [24]:
epochs = 10
batch_size = 32

In [25]:
history = model.fit(
    train_datagen.flow(X, y, batch_size=batch_size),
    validation_data=(X_val, y_val),
    steps_per_epoch=len(X) // batch_size,
    epochs=epochs,
    verbose=1
)


Epoch 1/10
333/333 [==============================] - 124s 354ms/step - loss: 0.5026 - accuracy: 0.7791 - val_loss: 0.7038 - val_accuracy: 0.6658
Epoch 2/10
333/333 [==============================] - 114s 344ms/step - loss: 0.4802 - accuracy: 0.7882 - val_loss: 0.7313 - val_accuracy: 0.6583
Epoch 3/10
333/333 [==============================] - 114s 343ms/step - loss: 0.4680 - accuracy: 0.7943 - val_loss: 0.7016 - val_accuracy: 0.6642
Epoch 4/10
333/333 [==============================] - 115s 347ms/step - loss: 0.4641 - accuracy: 0.7981 - val_loss: 0.7737 - val_accuracy: 0.6620
Epoch 5/10
333/333 [==============================] - 113s 340ms/step - loss: 0.4593 - accuracy: 0.7989 - val_loss: 0.8558 - val_accuracy: 0.6631
Epoch 6/10
333/333 [==============================] - 114s 343ms/step - loss: 0.4508 - accuracy: 0.8029 - val_loss: 0.7756 - val_accuracy: 0.6668
Epoch 7/10
333/333 [==============================] - 111s 335ms/step - loss: 0.4524 - accuracy: 0.8022 - val_loss: 0.8328 -

In [26]:
model.save('mobilnetv2_leukemia_model.h5')

In [27]:
from tensorflow.keras.models import load_model

model = load_model('mobilnetv2_leukemia_model.h5')

val_loss, val_accuracy = model.evaluate(X_val, y_val)

print(f"Validation Accuracy: {val_accuracy * 100:.2f}%")

59/59 [==============================] - 13s 193ms/step - loss: 0.7673 - accuracy: 0.6685
Validation Accuracy: 66.85%
